In [3]:

import sys
from pathlib import Path

In [4]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset

# Xử lý dữ liệu

## Lấy dữ liệu

Lấy dữ liệu từ file CSV

In [5]:
# Lấy dữ liệu từ nguồn sẵn có
import sys
import os
import talib as ta
sys.path.append('d:\\NCKH\\SARSA_FinancialRL')  # Thêm đường dẫn gốc để import module

import pandas as pd
from data.data_processor.feature_engineer import engineer_stat as es
from data.split_data.split import split_data_two_parts
from environments.stock_trading_env.mdp import StockTradingMDP as stockMDP
from agents.d_sarsa.d_sarsa import train_deep_sarsa

# Định nghĩa đường dẫn gốc
base_path = 'd:\\NCKH\\SARSA_FinancialRL'

# Lấy dữ liệu tạm từ nguồn sẵn có
csv_path = os.path.join(base_path, 'data', 'data_storer', 'data_research', 'FPT_data.csv')
df = pd.read_csv(csv_path)
df

,time,open,high,low,close,volume
0,20/06/2012,5.31,5.31,5.27,5.29,89720
1,21/06/2012,5.28,5.29,5.26,5.28,50360
2,22/06/2012,5.27,5.28,5.26,5.26,67350
3,25/06/2012,5.25,5.29,5.25,5.26,42840
4,26/06/2012,5.22,5.24,5.22,5.22,31110
...,...,...,...,...,...,...
3128,25/12/2024,129.85,130.37,129.68,130.02,2517969
3129,26/12/2024,130.19,130.54,128.56,129.16,2238343
3130,27/12/2024,128.90,129.25,128.21,128.99,2567003
3131,30/12/2024,128.81,130.02,128.04,129.76,1986819


## Tính toán các chỉ báo kĩ thuật

In [6]:
df['time'] = pd.to_datetime(df['time'])
df = df.sort_values('time').reset_index(drop=True)

# Gọi hàm với start_date và config tùy chỉnh
start_date = pd.to_datetime('01-01-2013', format='%d-%m-%Y')
price_history = es.add_technical_indicators(
    df, 
    start_date=start_date
)
# df_with_indicators giờ đã có các cột chỉ số, gắn vào df gốc của bạn
price_history

C:\Users\Admin\AppData\Local\Temp\ipykernel_9136\1549584949.py:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['time'] = pd.to_datetime(df['time'])


,time,open,high,low,close,volume,MACD,RSI,CCI,ADX
0,02/01/2013,4.00,4.15,4.00,4.13,380320,0.062248,72.002503,253.212693,26.054096
1,03/01/2013,4.13,4.15,4.00,4.07,366960,0.073297,66.503579,180.086580,28.047868
2,04/01/2013,4.05,4.13,4.04,4.13,278020,0.085905,69.049162,156.238566,29.899227
3,07/01/2013,4.11,4.14,4.08,4.11,204570,0.093209,67.215490,133.887463,31.676318
4,08/01/2013,4.12,4.31,4.11,4.31,535860,0.113823,74.506365,161.115250,34.114683
...,...,...,...,...,...,...,...,...,...,...
2991,25/12/2024,129.85,130.37,129.68,130.02,2517969,2.858353,67.076480,80.770363,34.110079
2992,26/12/2024,130.19,130.54,128.56,129.16,2238343,2.718885,63.212308,61.673685,33.596240
2993,27/12/2024,128.90,129.25,128.21,128.99,2567003,2.565070,62.446483,34.093919,32.865699
2994,30/12/2024,128.81,130.02,128.04,129.76,1986819,2.476752,64.541897,51.584632,32.527327


## Chia dữ liệu

In [7]:
# Chia dữ liệu theo khoảng thời gian cụ thể
price_history['time'] = pd.to_datetime(price_history['time'], format='%d/%m/%Y')

# Good period
good_train_FPT = price_history[(price_history['time'] >= '2013-01-01') & (price_history['time'] <= '2018-12-31')]
good_test_FPT = price_history[(price_history['time'] >= '2019-01-01') & (price_history['time'] <= '2021-12-31')]

# Bad period
bad_train_FPT = price_history[(price_history['time'] >= '2013-01-01') & (price_history['time'] <= '2021-12-31')]
bad_test_FPT = price_history[(price_history['time'] >= '2022-01-01') & (price_history['time'] <= '2023-12-31')]

print(f"Good Train FPT: {len(good_train_FPT)} rows")
good_train_FPT
print(f"Good Test FPT: {len(good_test_FPT)} rows")
good_test_FPT
print(f"Bad Train FPT: {len(bad_train_FPT)} rows")
bad_train_FPT
print(f"Bad Test FPT: {len(bad_test_FPT)} rows")
bad_test_FPT




Good Train FPT: 1496 rows
Good Test FPT: 752 rows
Bad Train FPT: 2248 rows
Bad Test FPT: 498 rows


,time,open,high,low,close,volume,MACD,RSI,CCI,ADX
2248,2022-01-04,47.90,48.31,47.85,47.95,1935000,-0.429141,41.223834,-98.765432,15.598851
2249,2022-01-05,48.05,48.31,47.95,48.00,1193800,-0.422749,41.848098,-81.826209,16.484579
2250,2022-01-06,48.00,48.21,47.95,47.95,1192400,-0.416912,41.374851,-79.153439,17.307042
2251,2022-01-07,47.75,48.05,47.75,47.90,1016800,-0.411577,40.877026,-88.142730,18.430113
2252,2022-01-10,47.80,47.95,47.13,47.18,1882400,-0.460142,34.449168,-136.667092,20.377769
...,...,...,...,...,...,...,...,...,...,...
2741,2023-12-25,70.18,71.14,70.10,70.99,2019931,0.662247,58.868587,79.992576,16.962281
2742,2023-12-26,71.36,72.91,71.28,71.88,3364283,0.749914,62.726163,179.664883,18.727718
2743,2023-12-27,72.02,72.32,71.65,71.65,1359748,0.791705,61.130571,151.077958,20.367053
2744,2023-12-28,71.73,71.73,71.21,71.43,1199164,0.797875,59.569661,101.561551,21.275093


In [8]:
# Lưu các biến DataFrame thành file CSV
import os

# Thư mục train
train_dir = r'D:\nckh\SARSA_FinancialRL\data\data_storer\data_research\train'
os.makedirs(train_dir, exist_ok=True)
good_train_FPT.to_csv(os.path.join(train_dir, 'good_train_FPT.csv'), index=False)
bad_train_FPT.to_csv(os.path.join(train_dir, 'bad_train_FPT.csv'), index=False)

# Thư mục test
test_dir = r'D:\nckh\SARSA_FinancialRL\data\data_storer\data_research\test'
os.makedirs(test_dir, exist_ok=True)
good_test_FPT.to_csv(os.path.join(test_dir, 'good_test_FPT.csv'), index=False)
bad_test_FPT.to_csv(os.path.join(test_dir, 'bad_test_FPT.csv'), index=False)

print("Đã lưu tất cả file CSV thành công!")


Đã lưu tất cả file CSV thành công!


# Triển khai chính sách ngẫu nhiên
làm tiên chuẩn (baseline) để so sánh với kết quả với kết quả của ba agent sắp tới.

Chính sách này không học hỏi gì, chỉ chọn hành động ngẫu nhiên đều đặn, như tung xúc xắc để quyết định mua/bán.

Chạy trên tập test

In [9]:
train = good_train_FPT
test = good_test_FPT
# Định nghĩa danh sách hành động (A) - cần thiết cho pi_random
balance_init = 1000  # số dư ban đầu
k = 5  # số cổ phiếu tối đa
min_balance = -100  # ngưỡng số dư tối thiểu
A = [a for a in range(-k, k + 1, 1)]  # hành động: -5 đến +5

# Tạo instance của StockTradingMDP
mdp = stockMDP(balance_init=balance_init, k=k, min_balance=min_balance)

# policy
def pi_random(s, greedy=False, eps=0.2):
    return np.random.choice(A) # random policy (uniform distributed)

profits = list()
for rep in tqdm(range(20)):
    profit = mdp.interact_test(
        pi=pi_random,  # chính sách (hàm pi_random)
        train_series=train,  # DataFrame train (từ cell chia dữ liệu)
        test_series=test,  # DataFrame test (từ cell chia dữ liệu)
        series_name='test',  # chạy trên tập test
        verbose=False  # không hiển thị biểu đồ (chỉ thu thập dữ liệu)
    )
    profits.append(profit)
print("Avg Profit", np.mean(profits))
print('STD Profit', np.std(profits))

100%|██████████| 20/20 [00:01<00:00, 13.02it/s]

Avg Profit 960.0596100000005
STD Profit 399.4590434734541


In [10]:
good_train_FPT

,time,open,high,low,close,volume,MACD,RSI,CCI,ADX
0,2013-01-02,4.00,4.15,4.00,4.13,380320,0.062248,72.002503,253.212693,26.054096
1,2013-01-03,4.13,4.15,4.00,4.07,366960,0.073297,66.503579,180.086580,28.047868
2,2013-01-04,4.05,4.13,4.04,4.13,278020,0.085905,69.049162,156.238566,29.899227
3,2013-01-07,4.11,4.14,4.08,4.11,204570,0.093209,67.215490,133.887463,31.676318
4,2013-01-08,4.12,4.31,4.11,4.31,535860,0.113823,74.506365,161.115250,34.114683
...,...,...,...,...,...,...,...,...,...,...
1491,2018-12-24,13.59,13.66,13.40,13.40,212530,-0.072050,39.937671,-88.370449,18.784244
1492,2018-12-25,13.21,13.24,13.08,13.24,732310,-0.103255,36.749962,-137.592868,19.396315
1493,2018-12-26,13.24,13.35,13.21,13.21,333590,-0.128920,36.167061,-113.548992,19.468731
1494,2018-12-27,13.43,13.48,13.34,13.34,454860,-0.137188,40.566302,-81.527716,18.977917


In [11]:
good_test_FPT

,time,open,high,low,close,volume,MACD,RSI,CCI,ADX
1496,2019-01-02,13.34,13.45,13.13,13.15,459120,-0.152099,37.659421,-90.424646,18.775608
1497,2019-01-03,13.23,13.23,12.99,13.02,716600,-0.176069,35.249637,-107.031598,19.412826
1498,2019-01-04,12.95,13.23,12.92,13.11,318570,-0.185662,38.198065,-97.740113,20.197787
1499,2019-01-07,13.34,13.43,13.27,13.34,408470,-0.172715,45.080538,-49.440354,19.964952
1500,2019-01-08,13.34,13.40,13.31,13.40,295660,-0.155816,46.746625,-41.258836,19.748747
...,...,...,...,...,...,...,...,...,...,...
2243,2021-12-27,48.31,48.72,47.85,48.47,793700,-0.340125,44.216418,-126.648513,11.999769
2244,2021-12-28,48.47,48.67,48.26,48.47,567300,-0.337767,44.216418,-95.173115,12.267674
2245,2021-12-29,48.62,48.62,48.00,48.00,912200,-0.369564,40.508430,-121.615087,12.927191
2246,2021-12-30,48.05,48.36,47.95,48.00,810800,-0.390264,40.508430,-121.320795,13.617754


# Huấn luyện agent Deep - SARSA với tình huống tốt


In [14]:
"""
shared_config = {
    'episodes': 30,         # Original episodes
    'gamma': 0.6,          # Original gamma
    'alpha': 0.7,           # Original alpha
    'epsilon_start': 0.8,   # Original epsilon_start
    'epsilon_min': 0.2,     # Original epsilon_min
    'epsilon_decay': 0.9,  # Chỉnh nhẹ từ 0.98 xuống 0.95 để khám phá chậm hơn trên FPT
    'nn_epochs': 10,        # Original nn_epochs
    'nn_lr': 1e-5,          # Original nn_lr
    'enabled_agents': [1, 1,1]  # [SARSA, DQN, PG]: 1 để bật, 0 để tắt
}
"""
shared_config = {
    'episodes': 35,         # Original episodes
    'gamma': 0.95,          # Original gamma
    'alpha': 0.6,           # Original alpha
    'epsilon_start': 0.9,   # Original epsilon_start
    'epsilon_min': 0.1,     # Original epsilon_min
    'epsilon_decay': 0.95,  # Chỉnh nhẹ từ 0.98 xuống 0.95 để khám phá chậm hơn trên FPT
    'nn_epochs': 10,        # Original nn_epochs
    'nn_lr': 1e-5,          # Original nn_lr
    'enabled_agents': [1, 1,1]  # [SARSA, DQN, PG]: 1 để bật, 0 để tắt
}

"""'episodes': 35, 'gamma': 0.8369349814523732, 'alpha': 0.5680671593345815, 'epsilon_start': 0.9540941536350933, 'epsilon_min': 0.19785221090168595, 'epsilon_decay': 0.9489652036913004, 'nn_epochs': 32, 'nn_lr': 0.0002990739041169814}"""

"'episodes': 35, 'gamma': 0.8369349814523732, 'alpha': 0.5680671593345815, 'epsilon_start': 0.9540941536350933, 'epsilon_min': 0.19785221090168595, 'epsilon_decay': 0.9489652036913004, 'nn_epochs': 32, 'nn_lr': 0.0002990739041169814}"

In [17]:
# Thêm đường dẫn gốc để import module
import sys
sys.path.append('d:\\NCKH\\SARSA_FinancialRL')

# Import run_experiments từ agents.run
from agents.run import run_experiments

# Định nghĩa shared_config với original settings từ paper, chỉnh nhẹ epsilon_decay xuống 0.95 để phù hợp với FPT data


# Chuyển lại Good Period
train = good_train_FPT
test = good_test_FPT

# Chạy thử nghiệm cho agent được bật với cùng siêu tham số
results = run_experiments(mdp, train, test, shared_config, num_runs=20)

# Hiển thị kết quả
import matplotlib.pyplot as plt

# Vẽ learning curves cho từng agent
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

agents = ['sarsa', 'dqn', 'pg']
titles = ['Deep SARSA', 'DQN', 'Policy Gradient']

for i, (agent, title) in enumerate(zip(agents, titles)):
    if agent in results:
        res = results[agent]
        mean_curve = res['learning_curve']
        
        axes[i].plot(mean_curve, label='Mean Profit')
        axes[i].set_title(f'{title} Learning Curve')
        axes[i].set_xlabel('Episode')
        axes[i].set_ylabel('Profit ($)')
        axes[i].legend()
        axes[i].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Định nghĩa hàm tính Annual Return Rate
def agent_annual_return(initial_capital, final_portfolio, start_date, end_date):
    total_return = (final_portfolio / initial_capital) - 1
    years = (pd.to_datetime(end_date) - pd.to_datetime(start_date)).days / 365.25
    if years <= 0: return 0, 0
    annual = (1 + total_return) ** (1 / years) - 1
    return annual * 100, total_return * 100

# Xác định start_date và end_date từ test_series
start_date = test.iloc[0]['time']
end_date = test.iloc[-1]['time']

# In tóm tắt kết quả
print("\n" + "="*80)
print("FINAL RESULTS SUMMARY")
print("="*80)

for agent in agents:
    if agent in results:
        res = results[agent]
        print(f"\n{res['agent']}:")
        print(f"  • Average Final Profit: ${res['final_profit']:.2f} ± ${res['std_final_profit']:.2f}")
        print(f"  • Best Profit: ${np.max(res['all_final_profits']):.2f}")
        print(f"  • Worst Profit: ${np.min(res['all_final_profits']):.2f}")
        if len(res['learning_curve']) > 0:
            print(f"  • Peak Training Profit: ${np.max(res['learning_curve']):.2f}")
        
        print(f"  • ROI: {res['roi']:.2f}%")
        
        # Tính Annual Return Rate
        final_portfolio = 1000 + res['final_profit']
        annual_rate, total_rate = agent_annual_return(1000, final_portfolio, start_date, end_date)
        print(f"  • Annual Return Rate: {annual_rate:.2f}%")
        print(f"  • Total Return Rate: {total_rate:.2f}%")

# So sánh với baseline (random policy)
print(f"\nRandom Policy Baseline:")
print(f"  • Average Profit: ${np.mean(profits):.2f} ± ${np.std(profits):.2f}")

# Tính Annual Return Rate cho baseline
baseline_final_portfolio = 1000 + np.mean(profits)
baseline_annual_rate, baseline_total_rate = agent_annual_return(1000, baseline_final_portfolio, start_date, end_date)
print(f"  • Annual Return Rate: {baseline_annual_rate:.2f}%")
print(f"  • Total Return Rate: {baseline_total_rate:.2f}%")


RUNNING DEEP SARSA EXPERIMENTS


Running Deep SARSA 20 times: 100%|██████████| 20/20 [10:20<00:00, 31.04s/it]



RUNNING DQN EXPERIMENTS


Running DQN 20 times: 100%|██████████| 20/20 [05:26<00:00, 16.35s/it]



RUNNING POLICY GRADIENT EXPERIMENTS


Running Policy Gradient 20 times: 100%|██████████| 20/20 [11:36<00:00, 34.80s/it]



EXPERIMENT SUMMARY

Deep SARSA:
  • Average Final Profit: $1523.30 ± $920.94
  • ROI: 152.33%
  • Annual Return Rate: 12.27%
  • Best Training Profit: $1666.20
  • Training Stability (std): $997.99

DQN:
  • Average Final Profit: $1058.04 ± $1166.41
  • ROI: 105.80%
  • Annual Return Rate: 9.44%
  • Best Training Profit: $1058.04
  • Training Stability (std): $1182.23

Policy Gradient:
  • Average Final Profit: $1751.34 ± $1169.81
  • ROI: 175.13%
  • Annual Return Rate: 13.49%



FINAL RESULTS SUMMARY

Deep SARSA:
  • Average Final Profit: $1523.30 ± $920.94
  • Best Profit: $2565.35
  • Worst Profit: $-0.68
  • Peak Training Profit: $1666.20
  • ROI: 152.33%
  • Annual Return Rate: 36.21%
  • Total Return Rate: 152.33%

DQN:
  • Average Final Profit: $1058.04 ± $1166.41
  • Best Profit: $2740.82
  • Worst Profit: $-1.67
  • Peak Training Profit: $1058.04
  • ROI: 105.80%
  • Annual Return Rate: 27.25%
  • Total Return Rate: 105.80%

Policy Gradient:
  • Average Final Profit: $1751.34 ± $1169.81
  • Best Profit: $2773.13
  • Worst Profit: $0.00
  • ROI: 175.13%
  • Annual Return Rate: 40.20%
  • Total Return Rate: 175.13%

Random Policy Baseline:
  • Average Profit: $960.06 ± $399.46
  • Annual Return Rate: 25.19%
  • Total Return Rate: 96.01%


In [27]:
# Vẽ biểu đồ Percentiles Over Repeated Experiments
import matplotlib.pyplot as plt
import numpy as np

# Danh sách percentiles để vẽ (từ 0% đến 100% với bước 5%)
percentile_levels = np.arange(0, 101, 5)

# Tạo figure
plt.figure(figsize=(12, 8))

# Vẽ cho từng agent
colors = ['blue', 'green', 'red']
agents = ['sarsa', 'dqn', 'pg']
agent_names = ['Deep SARSA', 'DQN', 'Policy Gradient']

for i, (agent, name) in enumerate(zip(agents, agent_names)):
    if agent in results:
        profits = results[agent]['all_final_profits']
        sorted_profits = np.sort(profits)
        percentile_values = np.percentile(sorted_profits, percentile_levels)
        plt.plot(percentile_levels, percentile_values, 
                color=colors[i], marker='o', linestyle='-', 
                label=name, linewidth=2, markersize=4)

# Thiết lập biểu đồ
plt.title('Độ biến thiên hiêu suất qua mỗi vòng lặp - FPT(good)', fontsize=16, fontweight='bold')
plt.xlabel('Percentile (%)', fontsize=14)
plt.ylabel('Profit ($)', fontsize=14)
plt.grid(True, alpha=0.3)
plt.legend(fontsize=12)
plt.xticks(percentile_levels)
plt.tight_layout()

# Hiển thị biểu đồ
plt.show()

In [29]:
# Vẽ đồ thị
portfolio_histories = {}
for agent_name in ['sarsa', 'dqn', 'pg']:
    if agent_name in results:
        portfolio_histories[agent_name] = results[agent_name]['portfolio_history']

plt.figure(figsize=(14, 8))
for agent, port in portfolio_histories.items():
    # Cắt portfolio_history để khớp với len(test['time'])
    plt.plot(test['time'], port[1:], label=agent.upper(), linewidth=2)

plt.title('Portfolio Value Over Time - FPT Good Period', fontsize=16, fontweight='bold')
plt.xlabel('Time', fontsize=14)
plt.ylabel('Portfolio Value ($)', fontsize=14)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [30]:
import os

models_dir = r'D:\nckh\SARSA_FinancialRL\models'

os.makedirs(models_dir, exist_ok=True)

# Save trained models for FPT good period
if 'sarsa' in results:
    torch.save(results['sarsa']['trained_agent'].state_dict(), os.path.join(models_dir, 'sarsa_good.pth'))
    print("Deep SARSA model saved.")

if 'dqn' in results:
    results['dqn']['trained_agent'].save(os.path.join(models_dir, 'dqn_good.pth'))
    print("DQN model saved.")

if 'pg' in results:
    results['pg']['trained_agent'].save_model(os.path.join(models_dir, 'pg_good.pth'))
    print("Policy Gradient model saved.")

print("All models for FPT good period saved successfully!")

Deep SARSA model saved.
DQN model saved.
Policy Gradient model saved.
All models for FPT good period saved successfully!


In [ ]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib import colors
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer, Image
from reportlab.lib.styles import getSampleStyleSheet
import os

# Đảm bảo thư mục tồn tại
results_dir = r'D:\nckh\SARSA_FinancialRL\application\results'
os.makedirs(results_dir, exist_ok=True)

# Tạo PDF
pdf_path = os.path.join(results_dir, 'FPT_good.pdf')
doc = SimpleDocTemplate(pdf_path, pagesize=letter)
elements = []
styles = getSampleStyleSheet()

# Title
elements.append(Paragraph("Training Results for FPT Good Period", styles['Title']))
elements.append(Spacer(1, 12))

# Hyperparameters
elements.append(Paragraph("Hyperparameters:", styles['Heading2']))
hp_text = f"""
Episodes: {shared_config['episodes']}
Gamma: {shared_config['gamma']}
Alpha: {shared_config['alpha']}
Epsilon Start: {shared_config['epsilon_start']}
Epsilon Min: {shared_config['epsilon_min']}
Epsilon Decay: {shared_config['epsilon_decay']}
NN Epochs: {shared_config['nn_epochs']}
NN LR: {shared_config['nn_lr']}
Enabled Agents: {shared_config['enabled_agents']}
"""
elements.append(Paragraph(hp_text, styles['Normal']))
elements.append(Spacer(1, 12))

# Results Table
elements.append(Paragraph("Training Results:", styles['Heading2']))

# Tạo data cho table
data = [['Agent', 'Average Final Profit', 'Best Profit', 'Worst Profit', 'Peak Training Profit', 'Annual Return Rate', 'Total Return Rate']]
for agent in agents:
    if agent in results:
        res = results[agent]
        avg_profit = f"${res['final_profit']:.2f} ± ${res['std_final_profit']:.2f}"
        best = f"${np.max(res['all_final_profits']):.2f}"
        worst = f"${np.min(res['all_final_profits']):.2f}"
        peak = f"${np.max(res['learning_curve']):.2f}" if len(res['learning_curve']) > 0 else 'N/A'
        final_portfolio = 1000 + res['final_profit']
        annual_rate, total_rate = agent_annual_return(1000, final_portfolio, start_date, end_date)
        annual = f"{annual_rate:.2f}%"
        total = f"{total_rate:.2f}%"
        data.append([res['agent'], avg_profit, best, worst, peak, annual, total])

# Baseline
baseline_avg = f"${np.mean(profits):.2f} ± ${np.std(profits):.2f}"
baseline_final_portfolio = 1000 + np.mean(profits)
baseline_annual_rate, baseline_total_rate = agent_annual_return(1000, baseline_final_portfolio, start_date, end_date)
data.append(['Random Policy Baseline', baseline_avg, 'N/A', 'N/A', 'N/A', f"{baseline_annual_rate:.2f}%", f"{baseline_total_rate:.2f}%"])

table = Table(data)
table.setStyle(TableStyle([
    ('BACKGROUND', (0,0), (-1,0), colors.grey),
    ('TEXTCOLOR', (0,0), (-1,0), colors.whitesmoke),
    ('ALIGN', (0,0), (-1,-1), 'CENTER'),
    ('FONTNAME', (0,0), (-1,0), 'Helvetica-Bold'),
    ('BOTTOMPADDING', (0,0), (-1,0), 12),
    ('BACKGROUND', (0,1), (-1,-1), colors.beige),
    ('GRID', (0,0), (-1,-1), 1, colors.black)
]))
elements.append(table)
elements.append(Spacer(1, 12))

# Biểu đồ Learning Curves
elements.append(Paragraph("Learning Curves:", styles['Heading2']))
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
for i, (agent, title) in enumerate(zip(agents, titles)):
    if agent in results:
        res = results[agent]
        mean_curve = res['learning_curve']
        axes[i].plot(mean_curve, label='Mean Profit')
        axes[i].set_title(f'{title} Learning Curve')
        axes[i].set_xlabel('Episode')
        axes[i].set_ylabel('Profit ($)')
        axes[i].legend()
        axes[i].grid(alpha=0.3)
plt.tight_layout()
learning_curve_path = os.path.join(results_dir, 'learning_curves.png')
plt.savefig(learning_curve_path)
plt.close()
elements.append(Image(learning_curve_path, width=500, height=300))

# Biểu đồ Percentiles
elements.append(Paragraph("Profit Percentiles:", styles['Heading2']))
plt.figure(figsize=(12, 8))
color_list = ['blue', 'green', 'red']
for i, (agent, name) in enumerate(zip(agents, agent_names)):
    if agent in results:
        agent_profits = results[agent]['all_final_profits']
        sorted_profits = np.sort(agent_profits)
        percentile_values = np.percentile(sorted_profits, percentile_levels)
        plt.plot(percentile_levels, percentile_values, color=color_list[i], marker='o', linestyle='-', label=name, linewidth=2, markersize=4)
plt.title('Độ biến thiên hiệu suất qua mỗi vòng lặp - FPT', fontsize=16, fontweight='bold')
plt.xlabel('Percentile (%)', fontsize=14)
plt.ylabel('Profit ($)', fontsize=14)
plt.grid(True, alpha=0.3)
plt.legend(fontsize=12)
plt.xticks(percentile_levels)
plt.tight_layout()
percentile_path = os.path.join(results_dir, 'percentiles.png')
plt.savefig(percentile_path)
plt.close()
elements.append(Image(percentile_path, width=500, height=400))

# Biểu đồ Portfolio Value Over Time
elements.append(Paragraph("Portfolio Value Over Time:", styles['Heading2']))
portfolio_histories = {}
for agent_name in ['sarsa', 'dqn', 'pg']:
    if agent_name in results:
        portfolio_histories[agent_name] = results[agent_name]['portfolio_history']

plt.figure(figsize=(14, 8))
for agent, port in portfolio_histories.items():
    plt.plot(test['time'], port, label=agent.upper(), linewidth=2)
plt.title('Portfolio Value Over Time - FPT Good Period', fontsize=16, fontweight='bold')
plt.xlabel('Time', fontsize=14)
plt.ylabel('Portfolio Value ($)', fontsize=14)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
portfolio_path = os.path.join(results_dir, 'portfolio_value_good.png')
plt.savefig(portfolio_path)
plt.close()
elements.append(Image(portfolio_path, width=500, height=400))

# Build PDF
doc.build(elements)
print(f"PDF saved to {pdf_path}")

ValueError: x and y must have same first dimension, but have shapes (752,) and (753,)

In [15]:
bad_train_FPT

,time,open,high,low,close,volume,MACD,RSI,CCI,ADX
0,2013-01-02,4.00,4.15,4.00,4.13,380320,0.062248,72.002503,253.212693,26.054096
1,2013-01-03,4.13,4.15,4.00,4.07,366960,0.073297,66.503579,180.086580,28.047868
2,2013-01-04,4.05,4.13,4.04,4.13,278020,0.085905,69.049162,156.238566,29.899227
3,2013-01-07,4.11,4.14,4.08,4.11,204570,0.093209,67.215490,133.887463,31.676318
4,2013-01-08,4.12,4.31,4.11,4.31,535860,0.113823,74.506365,161.115250,34.114683
...,...,...,...,...,...,...,...,...,...,...
1741,2019-12-25,20.59,21.14,20.59,20.99,1604350,-0.039673,57.065834,135.698045,22.959103
1742,2019-12-26,21.10,21.21,20.95,20.95,1124500,-0.006003,56.231303,148.692609,22.272090
1743,2019-12-27,20.95,21.10,20.73,21.03,1004860,0.026826,57.567826,114.976959,20.944346
1744,2019-12-30,21.10,21.50,21.03,21.39,2881060,0.080959,63.037609,168.378331,20.695114


In [16]:
bad_test_FPT

,time,open,high,low,close,volume,MACD,RSI,CCI,ADX
2248,2022-01-04,47.90,48.31,47.85,47.95,1935000,-0.429141,41.223834,-98.765432,15.598851
2249,2022-01-05,48.05,48.31,47.95,48.00,1193800,-0.422749,41.848098,-81.826209,16.484579
2250,2022-01-06,48.00,48.21,47.95,47.95,1192400,-0.416912,41.374851,-79.153439,17.307042
2251,2022-01-07,47.75,48.05,47.75,47.90,1016800,-0.411577,40.877026,-88.142730,18.430113
2252,2022-01-10,47.80,47.95,47.13,47.18,1882400,-0.460142,34.449168,-136.667092,20.377769
...,...,...,...,...,...,...,...,...,...,...
2741,2023-12-25,70.18,71.14,70.10,70.99,2019931,0.662247,58.868587,79.992576,16.962281
2742,2023-12-26,71.36,72.91,71.28,71.88,3364283,0.749914,62.726163,179.664883,18.727718
2743,2023-12-27,72.02,72.32,71.65,71.65,1359748,0.791705,61.130571,151.077958,20.367053
2744,2023-12-28,71.73,71.73,71.21,71.43,1199164,0.797875,59.569661,101.561551,21.275093


# Huấn luyện trên trường hợp xấu 

In [ ]:
# Import run_experiments từ agents.run
from agents.run import run_experiments



# Chuyển lại Good Period
train = bad_train_FPT
test = bad_test_FPT

# Chạy thử nghiệm cho agent được bật với cùng siêu tham số
results = run_experiments(mdp, train, test, shared_config, num_runs=20)

# Hiển thị kết quả
import matplotlib.pyplot as plt

# Vẽ learning curves cho từng agent
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

agents = ['sarsa', 'dqn', 'pg']
titles = ['Deep SARSA', 'DQN', 'Policy Gradient']

for i, (agent, title) in enumerate(zip(agents, titles)):
    if agent in results:
        res = results[agent]
        mean_curve = res['learning_curve']
        
        axes[i].plot(mean_curve, label='Mean Profit')
        axes[i].set_title(f'{title} Learning Curve')
        axes[i].set_xlabel('Episode')
        axes[i].set_ylabel('Profit ($)')
        axes[i].legend()
        axes[i].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Thêm ba biểu đồ riêng lẻ cho từng thuật toán
for i, (agent, title) in enumerate(zip(agents, titles)):
    if agent in results:
        res = results[agent]
        mean_curve = res['learning_curve']
        
        plt.figure(figsize=(8, 5))
        plt.plot(mean_curve, label='Mean Profit', color='blue')
        plt.title(f'{title} Learning Curve (Individual)')
        plt.xlabel('Episode')
        plt.ylabel('Profit ($)')
        plt.legend()
        plt.grid(alpha=0.3)
        plt.show()

# Định nghĩa hàm tính Annual Return Rate
def agent_annual_return(initial_capital, final_portfolio, start_date, end_date):
    total_return = (final_portfolio / initial_capital) - 1
    years = (pd.to_datetime(end_date) - pd.to_datetime(start_date)).days / 365.25
    if years <= 0: return 0, 0
    annual = (1 + total_return) ** (1 / years) - 1
    return annual * 100, total_return * 100

# Xác định start_date và end_date từ test_series
start_date = test.iloc[0]['time']
end_date = test.iloc[-1]['time']

# In tóm tắt kết quả
print("\n" + "="*80)
print("FINAL RESULTS SUMMARY")
print("="*80)

for agent in agents:
    if agent in results:
        res = results[agent]
        print(f"\n{res['agent']}:")
        print(f"  • Average Final Profit: ${res['final_profit']:.2f} ± ${res['std_final_profit']:.2f}")
        print(f"  • Best Profit: ${np.max(res['all_final_profits']):.2f}")
        print(f"  • Worst Profit: ${np.min(res['all_final_profits']):.2f}")
        if len(res['learning_curve']) > 0:
            print(f"  • Peak Training Profit: ${np.max(res['learning_curve']):.2f}")
        
        # Tính Annual Return Rate
        final_portfolio = 1000 + res['final_profit']
        annual_rate, total_rate = agent_annual_return(1000, final_portfolio, start_date, end_date)
        print(f"  • Annual Return Rate: {annual_rate:.2f}%")
        print(f"  • Total Return Rate: {total_rate:.2f}%")
    else:
        print(f"\n{agent.upper()}: No results available (possibly training failed)")

# So sánh với baseline (random policy)
print(f"\nRandom Policy Baseline:")
print(f"  • Average Profit: ${np.mean(profits):.2f} ± ${np.std(profits):.2f}")

# Tính Annual Return Rate cho baseline
baseline_final_portfolio = 1000 + np.mean(profits)
baseline_annual_rate, baseline_total_rate = agent_annual_return(1000, baseline_final_portfolio, start_date, end_date)
print(f"  • Annual Return Rate: {baseline_annual_rate:.2f}%")
print(f"  • Total Return Rate: {baseline_total_rate:.2f}%")


RUNNING DEEP SARSA EXPERIMENTS


Running Deep SARSA 20 times: 100%|██████████| 20/20 [12:55<00:00, 38.79s/it]



RUNNING DQN EXPERIMENTS


Running DQN 20 times: 100%|██████████| 20/20 [06:06<00:00, 18.31s/it]



RUNNING POLICY GRADIENT EXPERIMENTS


Running Policy Gradient 20 times: 100%|██████████| 20/20 [12:20<00:00, 37.04s/it]



EXPERIMENT SUMMARY

Deep SARSA:
  • Average Final Profit: $360.95 ± $245.05
  • Annual Return Rate: 3.93%
  • Best Training Profit: $448.95
  • Training Stability (std): $242.97

DQN:
  • Average Final Profit: $388.86 ± $210.59
  • Annual Return Rate: 4.19%
  • Best Training Profit: $392.89
  • Training Stability (std): $212.34

Policy Gradient:
  • Average Final Profit: $254.74 ± $247.94
  • Annual Return Rate: 2.88%



FINAL RESULTS SUMMARY

Deep SARSA:
  • Average Final Profit: $360.95 ± $245.05
  • Best Profit: $742.80
  • Worst Profit: $-43.23
  • Peak Training Profit: $448.95
  • Annual Return Rate: 16.82%
  • Total Return Rate: 36.10%

DQN:
  • Average Final Profit: $388.86 ± $210.59
  • Best Profit: $588.69
  • Worst Profit: $-9.74
  • Peak Training Profit: $392.89
  • Annual Return Rate: 18.02%
  • Total Return Rate: 38.89%

Policy Gradient:
  • Average Final Profit: $254.74 ± $247.94
  • Best Profit: $629.80
  • Worst Profit: $0.00
  • Annual Return Rate: 12.13%
  • Total Return Rate: 25.47%

Random Policy Baseline:
  • Average Profit: $839.14 ± $1108.39
  • Annual Return Rate: 35.99%
  • Total Return Rate: 83.91%


In [18]:
# Vẽ biểu đồ Percentiles Over Repeated Experiments
import matplotlib.pyplot as plt
import numpy as np

# Danh sách percentiles để vẽ (từ 0% đến 100% với bước 5%)
percentile_levels = np.arange(0, 101, 5)

# Tạo figure
plt.figure(figsize=(12, 8))

# Vẽ cho từng agent
colors = ['blue', 'green', 'red']
agents = ['sarsa', 'dqn', 'pg']
agent_names = ['Deep SARSA', 'DQN', 'Policy Gradient']

for i, (agent, name) in enumerate(zip(agents, agent_names)):
    if agent in results:
        profits = results[agent]['all_final_profits']
        sorted_profits = np.sort(profits)
        percentile_values = np.percentile(sorted_profits, percentile_levels)
        plt.plot(percentile_levels, percentile_values, 
                color=colors[i], marker='o', linestyle='-', 
                label=name, linewidth=2, markersize=4)

# Thiết lập biểu đồ
plt.title('Độ biến thiên hiêu suất qua mỗi vòng lặp - FPT(bad)', fontsize=16, fontweight='bold')
plt.xlabel('Percentile (%)', fontsize=14)
plt.ylabel('Profit ($)', fontsize=14)
plt.grid(True, alpha=0.3)
plt.legend(fontsize=12)
plt.xticks(percentile_levels)
plt.tight_layout()

# Hiển thị biểu đồ
plt.show()

In [25]:
# Vẽ đồ thị Portfolio Value Over Time cho Bad Period
import matplotlib.pyplot as plt

portfolio_histories = {}

for agent_name in ['sarsa', 'dqn', 'pg']:
    if agent_name in results:
        agent = results[agent_name]['trained_agent']
        
        # Khởi tạo trạng thái từ cuối tập train
        prev_row = train.iloc[-1]
        state_init = [
            float(prev_row['close']), 
            mdp.balance_init, 
            0,
            float(prev_row['MACD']),
            float(prev_row['RSI']),
            float(prev_row['CCI']),
            float(prev_row['ADX'])
        ]
        
        # Chạy simulation trên tập test
        states, _, _ = mdp.simulate(test, state_init, agent.pi, True)
        
        # Tính portfolio value tại mỗi bước
        portfolio = [s[1] + s[0] * s[2] for s in states]
        portfolio_histories[agent_name] = portfolio

# Vẽ đồ thị
plt.figure(figsize=(14, 8))
for agent, port in portfolio_histories.items():
    plt.plot(test['time'], port, label=agent.upper(), linewidth=2)

plt.title('Portfolio Value Over Time - FPT Bad Period', fontsize=16, fontweight='bold')
plt.xlabel('Time', fontsize=14)
plt.ylabel('Portfolio Value ($)', fontsize=14)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Tính và hiển thị ROI cho từng agent
print("\n" + "="*60)
print("RETURN ON INVESTMENT (ROI) SUMMARY - BAD PERIOD")
print("="*60)

initial_investment = mdp.balance_init
for agent in agents:
    if agent in results:
        res = results[agent]
        print(f"{res['agent']}: ROI = {res['roi']:.2f}%")

# ROI cho baseline
baseline_final_portfolio = initial_investment + np.mean(profits)
baseline_roi = ((baseline_final_portfolio - initial_investment) / initial_investment) * 100
print(f"Random Policy Baseline: ROI = {baseline_roi:.2f}%")

AttributeError: 'Qsa' object has no attribute 'pi'

In [19]:
import os

models_dir = r'D:\nckh\SARSA_FinancialRL\models'

os.makedirs(models_dir, exist_ok=True)

# Save trained models for FPT bad period
if 'sarsa' in results:
    torch.save(results['sarsa']['trained_agent'].state_dict(), os.path.join(models_dir, 'sarsa_bad.pth'))
    print("Deep SARSA model saved.")

if 'dqn' in results:
    results['dqn']['trained_agent'].save(os.path.join(models_dir, 'dqn_bad.pth'))
    print("DQN model saved.")

if 'pg' in results:
    results['pg']['trained_agent'].save_model(os.path.join(models_dir, 'pg_bad.pth'))
    print("Policy Gradient model saved.")

print("All models for FPT bad period saved successfully!")

Deep SARSA model saved.
DQN model saved.
Policy Gradient model saved.
All models for FPT bad period saved successfully!


In [ ]:
# Biểu đồ Portfolio Value Over Time
elements.append(Paragraph("Portfolio Value Over Time:", styles['Heading2']))
portfolio_histories = {}
for agent_name in ['sarsa', 'dqn', 'pg']:
    if agent_name in results:
        portfolio_histories[agent_name] = results[agent_name]['portfolio_history']

plt.figure(figsize=(14, 8))
for agent, port in portfolio_histories.items():
    plt.plot(test['time'], port, label=agent.upper(), linewidth=2)
plt.title('Portfolio Value Over Time - FPT Bad Period', fontsize=16, fontweight='bold')
plt.xlabel('Time', fontsize=14)
plt.ylabel('Portfolio Value ($)', fontsize=14)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
portfolio_path = os.path.join(results_dir, 'portfolio_value_bad.png')
plt.savefig(portfolio_path)
plt.close()
elements.append(Image(portfolio_path, width=500, height=400))

NameError: name 'results_dir' is not defined

NameError: name 'results_dir' is not defined